In [60]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# Load the Dataset
deal = pd.read_csv('/content/drive/MyDrive/데청캠_인바디/ZohoCRM_Dataset/deal_closed_prediction_data.csv')

print(deal)

X = deal[['Year', 'Month', 'Day', 'LeadSource', 'Territory', 'Industry']]
y = deal[['Stage']]

print(y.value_counts())

# StandarScaler 적용
# scaler = StandardScaler()
# scaler.fit(X)
# X_scaled = scaler.transform(X)

# Min-Max Scaling
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X) 
scaled = scaler.transform(X) 

      Year  Month  Day  LeadSource    Territory       Industry  Stage
0     2017      2    7    0.000458    50.000000  132320.620804      0
1     2017      2    7    0.000458    50.000000  132320.620804      0
2     2017      2    7    0.000458  3430.305858  132320.620804      1
3     2017      2    7    0.000458  1394.639984   61963.087644      1
4     2017      2   14    0.000458    50.000000  132320.620804      1
...    ...    ...  ...         ...          ...            ...    ...
2136  2021      7   24   32.692308   645.756033    1430.564418      0
2137  2021      7   26   25.000000   645.756033    1430.564418      0
2138  2021      7   26   25.000000  1014.059096    2727.317659      0
2139  2021      7   26   40.384615   645.756033    2727.317659      0
2140  2021      7   26   32.692308   645.756033    1073.495543      0

[2141 rows x 7 columns]
Stage
0        1212
1         929
dtype: int64


In [62]:
# Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled , y, test_size=0.3, random_state=1)

print(X_train.shape)
print(y_test.shape)

(1498, 6)
(643, 1)


In [80]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV


# Test1 : 선형 분리(linear) -  80.64085447263018 %

parameters = {'kernel':['rbf','linear','poly'],
              'C': [.1,.05,1, 10,15,20, 25,50],
              'gamma': [.1,.2,.3,.05,.001]}

model = svm.SVC(kernel = 'rbf',verbose=0)

grid_svm = GridSearchCV(model,
                      param_grid = parameters, cv = 5)

grid_svm.fit(X_train, y_train.values.ravel())
print('Accuracy of Train : ',grid_svm.score(X_train,y_train)*100,'%')
# {'C': 1, 'gamma': 1}
# {'C': 15, 'gamma': 0.2, 'kernel': 'rbf'}
print('Best Params : ',grid_svm.best_params_)
pred_y = grid_svm.predict(X_test)

Accuracy of Train :  88.4512683578104 %
Best Params :  {'C': 15, 'gamma': 0.2, 'kernel': 'rbf'}


In [81]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

# 0.7107
# 0.8289
# 0.8398
# 0.8429
get_clf_eval(y_test, pred_y)

오차행렬:
 [[329  37]
 [ 64 213]]

정확도: 0.8429
정밀도: 0.8520
재현율: 0.7690
F1: 0.8083
AUC: 0.8339
